# Notebook 2 - Procesamiento de datos - v3

- v1: funciona, pero hay que revisar los datasets uno a uno de forma manual
- v2: análisis completo de todos los datasets y luego selección de los que interesan
- v3: limpieza de v2

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# conda install scipy -> versión 3.12 de Python, Scipy no funciona con la 3.13
from scipy.stats import skew

import owid.catalog as catalog

from typing import Dict, List, Optional, Union, Any

import ipywidgets as widgets
from IPython.display import display, clear_output
import asyncio

import time
from datetime import datetime

import concurrent.futures
import sys
import gc
import os

In [2]:
# versión de Python
print(f"{sys.version_info.major}.{sys.version_info.minor}")

3.12


In [3]:
# directorio para guardar los datos
dataframe_path = "..//data//dataframes"

## Funciones auxiliares

<b>1) Manejo de datos</b>

In [5]:
'''
v3 - añade URI única
'''
class DatasetManager:
    """
    Clase para gestionar eficientemente datasets de OWID, permitiendo cargas bajo demanda 
    y caching inteligente.
    """
    def __init__(self, cache_limit: int = 5):
        """
        Inicializa el gestor de datasets.
        
        Args:
            cache_limit: número máximo de datasets a mantener en memoria
        """
        # instancia del catálogo de Owid
        self.cat = catalog.RemoteCatalog()
        # lista de datasets que se pueden cargar correctamente
        self.successful_refs = []
        # lista de datasets que fallan al cargar
        self.failed_refs = []

        # caché para almacenar datasets cargados: {idx: table}
        self.cache = {}
        self.cache_limit = cache_limit
        # implementa estrategia LRU (Least Recently Used)
        self.cache_usage_order = []

        # DataFrame para guardar los datasets encontrados
        self.datasets_df = None

        # diccionarios para almacenar URIs de datasets cargados correctamente
        self.dataset_uris = {}  # {idx: uri_completa}
        self.dataset_uris_short = {}  # {idx: uri_abreviada}
    
    def find_covid_datasets(
        self, filtro: str = 'covid', public_only: bool = True
    ) -> pd.DataFrame:
        """
        Busca datasets relacionados con el filtro especificado.
        
        Args:
            filtro: str, término de búsqueda
            public_only: bool, si sólo se deben incluir datasets públicos
            
        Returns:
            DataFrame con los datasets filtrados
            Número de datasets públicos
        """
        # obtiene todos los datasets
        datasets = self.cat.find()
        print(f"Total de datasets encontrados: {len(datasets)}")
        
        # busca datasets por palabra clave
        print(f"Filtrando datasets que contienen '{filtro}'...")

        # filtra las columnas de texto (object)
        text_cols = datasets.select_dtypes(include=['object']).columns
        # máscara para seleccionar las filas que contienen el texto buscado
        filter_mask = False
        for col in text_cols:
            filter_mask |= datasets[col].str.contains(filtro, case=False, na=False)

        # aplica el filtro
        filtered_datasets = datasets[filter_mask]
        print(f"Datasets filtrados: {len(filtered_datasets)}")
        
        # selecciona sólo los datasets públicos si se ha solicitado
        if public_only:
            filtered_datasets = filtered_datasets[filtered_datasets["is_public"]]
            num_public_datasets = len(filtered_datasets)
            print(f"Total de datasets públicos: {num_public_datasets}")
        else:
            num_public_datasets = 0
        
        # guarda y devuelve los datasets filtrados
        self.datasets_df = filtered_datasets
        return filtered_datasets, num_public_datasets
    
    def test_datasets(self, max_count: int = None, timeout: int = 30) -> None:
        """
        Prueba la carga de cada dataset y almacena referencias a los cargados correctamente.
        
        Args:
            max_count: int, número máximo de datasets a probar (None = todos)
            timeout: int, tiempo máximo en segundos para intentar cargar cada dataset
        """
        if self.datasets_df is None:
            print("Primero debes buscar datasets con find_covid_datasets()")
            return
            
        # limpia las listas de referencias anteriores
        self.successful_refs = []
        self.failed_refs = []

        # Limpia URIs anteriores
        self.dataset_uris.clear()
        self.dataset_uris_short.clear()
        
        # inicia contadores
        count = 0
        total_success = 0
        total_failed = 0
        
        print(f"\nProbando datasets (con timeout de {timeout} segundos por dataset):")
        print("-" * 50)

        # recorre los datasets
        for idx, row in self.datasets_df.iterrows():
            if max_count is not None and count >= max_count:
                # alcanzado el número máximo de datasets a probar
                break
                
            count += 1
            # prueba el dataset actual
            success = self._test_load_dataset(idx, row, timeout)
            
            if success:
                # carga correcta
                self.successful_refs.append(idx)
                total_success += 1
            else:
                # la carga ha fallado
                self.failed_refs.append(idx)
                total_failed += 1
            
            # añade una pausa entre solicitudes
            time.sleep(1)
        
        print("-" * 50)
        print(f"Resumen: {total_success} datasets cargados correctamente, ",
              f"{total_failed} fallaron")
        print(f"Referencias almacenadas: {len(self.successful_refs)}")

    def _generate_uris(self, table) -> tuple[str, str]:
        """
        Genera las URIs completa y abreviada para un dataset.
    
        Args:
            table: objeto tabla cargado de OWID
        
        Returns:
            tuple: (uri_completa, uri_abreviada)
        """
        uri_completa = (
            f"data://{table.metadata.dataset.channel}"
            f"/{table.metadata.dataset.namespace}"
            f"/{table.metadata.dataset.version}"
            f"/{table.metadata.dataset.short_name}"
            f"/{table.metadata.short_name}"
        )
    
        uri_abreviada = (
            f"{table.metadata.dataset.short_name}"
            f"__{table.metadata.short_name}"
        )
    
        return uri_completa, uri_abreviada
    
    def _test_load_dataset(self, idx: str, row: pd.Series, timeout: int) -> bool:
        """
        Intenta cargar un dataset con manejo de timeout y errores.
        
        Args:
            idx: str, identificador del dataset
            row: Serie de Pandas, fila del DataFrame con información del dataset
            timeout: int, tiempo máximo en segundos para la carga
            
        Returns:
            bool: True si se cargó correctamente, False en caso contrario
        """
        try:
            print(f"Probando dataset {idx}", end="", flush=True)
            
            # implementa timeout usando un executor (carga en paralelo)
            with concurrent.futures.ThreadPoolExecutor() as executor:
                future = executor.submit(row.load)
                try:
                    table = future.result(timeout=timeout)
                    print(f" ✓ - Título: {table.metadata.dataset.title}")

                    # genera URIs
                    uri, uri_abreviada = self._generate_uris(table)
                    print(f" ✓ - URI única: {uri}")

                    # guarda URIs (se guardarán sólo si la carga es correcta)
                    self.dataset_uris[idx] = uri
                    self.dataset_uris_short[idx] = uri_abreviada
                    
                    # no almacena el dataset, sólo verifica que carga correctamente
                    
                    # Libera memoria inmediatamente
                    del table
                    gc.collect()
                    return True
                    
                except concurrent.futures.TimeoutError:
                    # se ha excedido el tiempo máximo para la carga
                    print(f" ✗ - TIMEOUT después de {timeout} segundos")
                    return False
                    
        except Exception as e:
            print(f" ✗ - ERROR: {str(e)}")
            return False
    
    def get_dataset(self, idx: str) -> Optional[Any]:
        """
        Obtiene un dataset por su identificador, usando caché si está disponible.
        Genera las URIs si no existen.
        
        Args:
            idx: str, identificador del dataset
            
        Returns:
            El dataset cargado o None si hay error
        """
        if self.datasets_df is None:
            print("Primero debes buscar datasets con find_covid_datasets()")
            return None

        # si el dataset ya está en caché, lo devuelve y actualiza el orden de uso
        if idx in self.cache:
            if idx in self.cache_usage_order:
                self.cache_usage_order.remove(idx)
            self.cache_usage_order.append(idx)

            # genera URIs si no existen (datasets cargados directamente sin test_datasets)
            if idx not in self.dataset_uris:
                uri, uri_abreviada = self._generate_uris(self.cache[idx])
                self.dataset_uris[idx] = uri
                self.dataset_uris_short[idx] = uri_abreviada
            
            return self.cache[idx]
        
        # si no está en caché, verifica que el dataset esé en la lista de carga correcta
        if idx not in self.successful_refs:
            print(f"El dataset {idx} no está en la lista de datasets cargados correctamente")
            return None
        
        # intenta cargar el dataset desde OWID
        try:
            print(f"Cargando dataset {idx}...", end="", flush=True)
            # carga el dataset
            table = self.datasets_df.loc[idx].load()
            print(" ✓")

            # genera URIs si no existen
            if idx not in self.dataset_uris:
                uri, uri_abreviada = self._generate_uris(table)
                self.dataset_uris[idx] = uri
                self.dataset_uris_short[idx] = uri_abreviada
            
            # gestiona la caché (estrategia LRU)
            if len(self.cache) >= self.cache_limit and self.cache_usage_order:
                # elimina el dataset usado menos recientemente
                oldest_idx = self.cache_usage_order.pop(0)
                del self.cache[oldest_idx]
                gc.collect()
            
            # añade el nuevo dataset a la caché
            self.cache[idx] = table
            self.cache_usage_order.append(idx)
            
            return table
            
        except Exception as e:
            print(f" ✗ - ERROR: {str(e)}")
            return None
    
    def clear_cache(self) -> None:
        """
        Libera toda la memoria de la caché
        """
        self.cache.clear()
        self.cache_usage_order.clear()
        self.dataset_uris.clear()
        self.dataset_uris_short.clear()
        
        gc.collect()
        print("Caché y URIs liberadas")
    
    def get_successful_refs(self) -> List[str]:
        """
        Devuelve las referencias a datasets cargados con éxito
        """
        return self.successful_refs.copy()
    
    def explore_dataset_structure(self, dataset) -> None:
        """
        Muestra información sobre la estructura de un dataset de OWID.
        Útil para entender cómo acceder a los datos.
        
        Args:
            dataset: el objeto dataset cargado por OWID
        """
        print(f"Tipo de objeto: {type(dataset)}")

        # muestra las URIs si están disponibles
        if hasattr(dataset, 'metadata'):
            uri_completa, uri_abreviada = self._generate_uris(dataset)
            print(f"\nURI única: {uri_completa}")
            print(f"URI abreviada: {uri_abreviada}")
        
        # muestra los metadatos si están disponibles
        if hasattr(dataset, 'metadata'):
            print("\nMetadatos:")
            meta = dataset.metadata
            
            if hasattr(meta, 'dataset'):
                ds_meta = meta.dataset
                # muestra el título si está disponible
                print(f"  - Título: {
                ds_meta.title if hasattr(ds_meta, 'title') else 'No disponible'
                }")
                # muestra los 100 primeros caracteres de la descripción si está disponible
                print(f"  - Descripción: {
                ds_meta.description[:100] + '...' if hasattr(
                    ds_meta, 'description'
                ) and ds_meta.description else 'No disponible'
                }")
            
        # intenta mostrar la estructura de los datos
        print("\nEstructura de datos:")
        try:
            # muestra dimensions si está disponible
            if hasattr(dataset, 'dimensions'):
                print(f"  - Dimensiones: {dataset.dimensions}")
            # muestra columns si está disponible
            if hasattr(dataset, 'columns'):
                print(f"  - Columnas: {dataset.columns}")
            # muestra (filas, columnas) si está disponible
            if hasattr(dataset, 'shape'):
                print(f"  - Forma: {dataset.shape}")
            # muestra la forma si tiene .values (como un array)
            if hasattr(dataset, 'values') and hasattr(dataset.values, 'shape'):
                print(f"  - Forma de valores: {dataset.values.shape}")
                
        except Exception as e:
            print(f"  - Error al acceder a la estructura: {e}")

    def get_dataset_uri(self, idx: str) -> Optional[str]:
        """
        Obtiene la URI completa de un dataset cargado correctamente.
        
        Args:
            idx: str, identificador del dataset
            
        Returns:
            str: URI completa o None si no existe
        """
        return self.dataset_uris.get(idx)

    def get_dataset_uri_short(self, idx: str) -> Optional[str]:
        """
        Obtiene la URI abreviada de un dataset cargado correctamente.
        
        Args:
            idx: str, identificador del dataset
            
        Returns:
            str: URI abreviada o None si no existe
        """
        return self.dataset_uris_short.get(idx)

    def get_all_uris(self) -> Dict[str, Dict[str, str]]:
        """
        Obtiene todas las URIs almacenadas.
        
        Returns:
            Dict con estructura: {idx: {'uri': uri_completa, 'uri_short': uri_abreviada}}
        """
        result = {}
        for idx in self.successful_refs:
            result[idx] = {
                'uri': self.dataset_uris.get(idx),
                'uri_short': self.dataset_uris_short.get(idx)
            }
        return result

In [6]:
def process_datasets(manager, processor_func=None, max_datasets=None):
    """
    Itera sobre todos los datasets cargados correctamente, cargándolos uno por uno.
    
    Args:
        manager: instancia de DatasetManager
        processor_func: función que recibe el dataset y su ID para procesarlo
            Si es None, sólo muestra información básica
        max_datasets: número máximo de datasets a procesar (None = todos)
    """
    # lista de IDs de datasets cargados correctamente
    successful_refs = manager.get_successful_refs()
    
    if not successful_refs:
        print("No hay datasets cargados correctamente para procesar.")
        return

    # calcula cuántos datasets se van a procesar
    total = len(successful_refs)
    if max_datasets is not None:
        total = min(total, max_datasets)
    
    print(f"Procesando {total} datasets de {len(successful_refs)} cargados correctamente:")

    # itera sobre los índices de los datasets a procesar
    for i, idx in enumerate(successful_refs[:total]):
        print(f"\n--- Dataset {i+1}/{total}: {idx} ---")
        
        # carga el dataset
        dataset = manager.get_dataset(idx)

        # comprueba que se haya cargado correctamente
        if dataset is not None:
            # si se ha proporcionado una función de procesamiento personalizada, la usa
            if processor_func:
                processor_func(dataset, idx)
            else:
                # muestra información básica del dataset
                print(f"Título: {
                dataset.metadata.dataset.title if hasattr(
                    dataset.metadata, 'dataset'
                ) and hasattr(dataset.metadata.dataset, 'title') else 'No disponible'
                }")

                # explora la estructura del dataset
                manager.explore_dataset_structure(dataset)
        
        # limpia la caché después de procesar cada dataset
        manager.clear_cache()

In [7]:
def show_info_dataset(references, manager, i):
    """
    Muestra los datos básicos del dataset i-ésimo.

    Args:
        referencies: lista de números de índice de los datasets
        manager: instancia del objeto DatasetManager
        i: int, número de orden del dataset de que queremos consultar la información

    Returns:
        dataset i-ésimo
    """
    # selecciona el dataset i-ésimo
    dataset_id = references[i]

    # carga el dataset
    dataset = manager.get_dataset(dataset_id)

    # muestra las URIs del dataset
    print("-" * 20, "URI ÚNICA", "-" * 20)
    print(manager.get_dataset_uri(dataset_id))
    print(f"URI ABREVIADA: {manager.get_dataset_uri_short(dataset_id)}")
    
    # muestra la información del dataset
    print("-" * 20, "INFO", "-" * 20)
    display(dataset.info())
    print("-" * 20, "COLUMNS", "-" * 20)
    display(dataset.columns)
    print("-" * 20, "METADATA", "-" * 20)
    display(dataset.metadata.to_dict())

    return dataset

In [8]:
def convert_to_pandas(dataset):
    """
    Convierte el dataset en un dataframe de Pandas

    Args:
        dataset: objeto Table de owid.catalog.catalogs.CatalogFrame

    Returns:
        dataframe de Pandas
    """
    # convierte owid.catalog.tables.Table en DataFrame puro de pandas
    df = pd.DataFrame(dataset)
    
    # convierte los índices de dataset en columnas normales del dataframe
    df = df.reset_index()

    # añade metadatos relevantes como columnas
    metadata = dataset.metadata.dataset
    df['dataset_title'] = metadata.title
    df['dataset_short_name'] = metadata.short_name
    df['dataset_description'] = metadata.description
    df['dataset_version'] = metadata.version
    df['dataset_update_period_days'] = metadata.update_period_days

    # muestra información del dataframe
    print(df.shape)
    display(df.head())

    return df

<b>2) Análisis de datos</b>

In [9]:
def get_dataset_title(dataset):
    """
    Busca el mejor título disponible en diferentes lugares del dataset

    Args:
        dataset: Dataframe de Pandas

    Returns:
        Lista de lugares donde buscar el título, por orden de preferencia
        
        titulos_posibles[0], 
        titulos_posibles[1] if len(titulos_posibles) > 1 else titulos_posibles[0]
    """
    # inicializa la lista de resultados
    titulos_posibles = []
    
    # 1. Título principal
    if hasattr(dataset.metadata, 'title') and dataset.metadata.title:
        titulos_posibles.append(dataset.metadata.title)
    
    # 2. Título del dataset
    if hasattr(dataset.metadata, 'dataset') and hasattr(
        dataset.metadata.dataset, 'title') and dataset.metadata.dataset.title:
        titulos_posibles.append(dataset.metadata.dataset.title)
    
    # 3. Nombre del dataset
    if hasattr(dataset.metadata, 'dataset') and hasattr(
        dataset.metadata.dataset, 'name') and dataset.metadata.dataset.name:
        titulos_posibles.append(dataset.metadata.dataset.name)
    
    # 4. Descripción corta
    if hasattr(dataset.metadata, 'description') and dataset.metadata.description:
        desc = dataset.metadata.description[:100] +\
        "..." if len(dataset.metadata.description) > 100 else dataset.metadata.description
        titulos_posibles.append(f"Desc: {desc}")
    
    # 5. Descripción del dataset
    if hasattr(dataset.metadata, 'dataset') and hasattr(
        dataset.metadata.dataset, 'description') and dataset.metadata.dataset.description:
        desc = dataset.metadata.dataset.description[:100] +\
        "..." if len(
            dataset.metadata.dataset.description
        ) > 100 else dataset.metadata.dataset.description
        titulos_posibles.append(f"Dataset desc: {desc}")
    
    # 6. ID o identificador
    if hasattr(dataset.metadata, 'id') and dataset.metadata.id:
        titulos_posibles.append(f"ID: {dataset.metadata.id}")
    
    # 7. Fallback
    if not titulos_posibles:
        titulos_posibles.append("Dataset sin título")
    
    return titulos_posibles[0], titulos_posibles[1] if len(
        titulos_posibles) > 1 else titulos_posibles[0]

In [10]:
def analyze_all_datasets(successful_refs, manager):
    """
    Muestra información relevante de todos los datasets

    Args:
        successful_refs: lista de referencias de datasets cargados correctamente
        manager: instancia del objeto DatasetManager

    Returns:
        lista con información de cada dataset
    """
    analisis_datasets = {}
    
    print("Analizando todos los datasets disponibles...")
    print("=" * 60)
    print(f"Total de datasets: {len(successful_refs)}")
    
    #print("\nPresiona Enter para ver cada dataset...")

    # recorre todos los datasets
    for i, ref in enumerate(successful_refs):
        
        # pausa para mostrar dataset por dataset
        #input(f"\n[Dataset {i+1}/{len(successful_refs)}] Presiona Enter para continuar...")

        print(f"\n🔍 DATASET {i+1}")
        print("=" * 50)

        # muestra la información del dataset
        dataset = show_info_dataset(successful_refs, manager, i)
        
        # obtiene el mejor título disponible
        titulo_principal, titulo_secundario = get_dataset_title(dataset)
        
        # convierte a dataframe de Pandas
        df = convert_to_pandas(dataset)
        
        ## busca fechas en cualquier columna que contenga 'date' o tipo Datetime
        fecha_min, fecha_max = None, None
        columnas_fecha = []

        # recorre todas las columnas
        for col in df.columns:
            if 'date' in col.lower() or df[col].dtype == 'datetime64[ns]':
                # encontrada columna con nombre que incluye la palabra 'date'
                # o que es de tipo Datetime
                columnas_fecha.append(col)
                try:
                    # verifica si la columna es de tipo Datetime
                    if pd.api.types.is_datetime64_any_dtype(df[col]):
                        # Datetime: fechas mínima y máxima
                        col_min = df[col].min()
                        col_max = df[col].max()
                        # actualiza fechas mínima y máxima extremas
                        if fecha_min is None or col_min < fecha_min:
                            fecha_min = col_min
                        if fecha_max is None or col_max > fecha_max:
                            fecha_max = col_max
                except:
                    pass
        
        ## muestra un resumen de los datos
        
        #print(f"\n🔍 DATASET {i+1}")
        #print("=" * 50)

        print("-" * 20, "RESUMEN DE LOS DATOS", "-" * 20)
        
        print(f"📊 Título principal: {titulo_principal}")
        if titulo_secundario != titulo_principal:
            print(f"📈 Título secundario: {titulo_secundario}")
        print(f"📏 Dimensiones: {len(df)} filas × {len(df.columns)} columnas")
        
        if columnas_fecha:
            print(f"📅 Columnas de fecha: {', '.join(columnas_fecha)}")
            if fecha_min and fecha_max:
                print(f"🗓️  Rango temporal: {fecha_min.strftime('%Y-%m-%d')} ",
                      f"→ {fecha_max.strftime('%Y-%m-%d')}")
        
        print(f"🏷️  Columnas: {', '.join(df.columns[:8])}",
              f"{'...' if len(df.columns) > 8 else ''}")
        
        # muestra las tres primeras filas de los datos
        print("\n📋 Vista previa (primeras 3 filas):")
        print(df.head(3).to_string(max_cols=6, max_colwidth=20))
        
        # información sobre tipos de datos
        tipos_datos = df.dtypes.value_counts()
        print(f"\n🔢 Tipos de datos: {dict(tipos_datos)}")

        # guarda un resumen del dataset
        uri_completa, uri_abreviada = manager._generate_uris(dataset)
        analisis_datasets[i+1] = {
            'dataset': dataset,
            'dataframe': df,
            'uri_unica': uri_completa,
            'uri_abreviada': uri_abreviada,
            'titulo': titulo_principal,
            'titulo_secundario': titulo_secundario,
            'num_filas': len(df),
            'num_columnas': len(df.columns),
            'columnas': list(df.columns),
            'columnas_fecha': columnas_fecha,
            'fecha_min': fecha_min,
            'fecha_max': fecha_max,
            'tipos_datos': dict(tipos_datos)
        }
    
    print(f"\n✅ Análisis completado. Se han procesado {len(successful_refs)} datasets.")

    # devuelve el resumen del dataset
    return analisis_datasets

<b>3) Guardado de datos</b>

In [11]:
'''
v2
'''
def select_datasets(datasets):
    """
    Permite que el usuario seleccione un grupo de datasets del conjunto completo.
    Muestra una lista de selección múltiple al usuario.
    
    Args:
        datasets: conjunto de datasets

    Returns:
        Diccionario con el conjunto de datasets seleccionados por el usuario
    """
    # inicializa la selección del usuario
    dataframes_seleccionados = {}

    # inicializa el resultado final
    resultado_final = {}

    ## opciones del selector
    
    # inicializa las opciones: tuplas (texto, num)
    opciones = []
    
    # recorre los datasets
    for num, info in datasets.items():

        # inicializa la información a mostrar
        linea = f"{num:2d}. "
        
        # URI abreviada
        uri_abreviada = info['uri_abreviada']
        linea += f"{uri_abreviada}"
        
        # título principal
        titulo_principal = info['titulo'] if info[
        'titulo'] != "Dataset sin título" else info['titulo_secundario']
        
        # limita la longitud del título principal a 50 caracteres
        if len(titulo_principal) > 50:
            titulo_principal = titulo_principal[:47] + "..."
        
        linea += f" - {titulo_principal}"

        # título secundario (sólo si es diferente del título principal)
        titulo_secundario = info['titulo_secundario'] if info[
        'titulo_secundario'] != titulo_principal else ""

        if titulo_secundario:
            # limita el título secundario a 60 caracteres
            if len(titulo_secundario) > 60:
                titulo_secundario = titulo_secundario[:57] + "..."
            linea += f" - {titulo_secundario}"

        # añade el texto como nueva opción de la lista de selección
        opciones.append((linea, num))

    ## widget de selección y botones
    
    # widget principal: lista de selección múltiple
    selector = widgets.SelectMultiple(
        options=opciones,
        description='',
        layout=widgets.Layout(width='600px', height='300px'),
        style={'description_width': 'initial'}
    )

    # botones de control: seleccionar todo, limpiar selección y procesar
    btn_todos = widgets.Button(
        description='Seleccionar Todos',
        button_style='info',
        layout=widgets.Layout(width='auto', margin='0 5px 0 0')
    )
    btn_ninguno = widgets.Button(
        description='Limpiar Selección',
        button_style='warning', 
        layout=widgets.Layout(width='auto', margin='0 5px 0 0')
    )
    btn_procesar = widgets.Button(
        description='✅ Procesar Selección',
        button_style='success',
        layout=widgets.Layout(width='auto', margin='0 5px 0 0')
    )

    # área de información
    info_label = widgets.HTML(value="<b>Seleccionados:</b> 0 datasets")
    
    # área de resultados
    resultado_area = widgets.Output()

    ## funciones internas
    
    def actualizar_info(change=None):
        """
        Actualiza el contador de seleccionados
        """
        count = len(selector.value)
        info_label.value = f"<b>Seleccionados:</b> {count} datasets"
    
    def seleccionar_todos(b):
        """
        Selecciona todos los datasets
        """
        selector.value = [num for _, num in opciones]
    
    def limpiar_seleccion(b):
        """
        Limpia la selección
        """
        selector.value = []

    def procesar_seleccion(b):
        """
        Procesa la selección y muestra los resultados.
        Guarda el resultado en 'resultado_final'.
        """
        nonlocal resultado_final
        
        with resultado_area:
            clear_output()
            
            numeros_seleccionados = list(selector.value)
            
            if not numeros_seleccionados:
                print("❌ No se ha seleccionado ningún dataset.")
                print("   Por favor, selecciona al menos un dataset antes de procesar.")
                return
            
            ## procesa la selección

            print("🔄 Procesando selección...")
            print("=" * 50)
            
            # inicializa la lista de dataframes seleccionados
            dataframes_seleccionados = {}
            
            print(f"\n📊 DATASETS SELECCIONADOS:")
            
            # recorre los datasets seleccionados
            for num in sorted(numeros_seleccionados):
                info = datasets[num]
                nombre_df = f'df_{num}'
                dataframes_seleccionados[nombre_df] = {
                    'dataframe': info['dataframe'],
                    'titulo': info['titulo'],
                    'descripcion': info['titulo_secundario'],
                    'uri_abreviada': info['uri_abreviada'],
                    'num_dataset': num,
                    'columnas': info['columnas'],
                    'num_filas': info['num_filas'],
                    'num_columnas': info['num_columnas']
                }
                    
                print(f"   ✅ Dataset {num}: {info['uri_abreviada']}")

            print(f"\n📊 RESUMEN FINAL:")
            print(f"   ✅ Datasets procesados: {len(numeros_seleccionados)}")

            # guarda el resultado
            resultado_final = dataframes_seleccionados

    # conecta botones y eventos con sus funciones correspondientes
    btn_todos.on_click(seleccionar_todos)
    btn_ninguno.on_click(limpiar_seleccion)
    btn_procesar.on_click(procesar_seleccion)
    selector.observe(actualizar_info, names='value')

    ## diseño de la interfaz
    
    # layout de botones
    botones_control = widgets.HBox([btn_todos, btn_ninguno, btn_procesar])

    # layout principal
    mensaje_titulo = "<h3>🔍 Selecciona los datasets que quieres procesar:</h3>"
    mensaje_ayuda = "<p><i>💡 Mantén presionado Ctrl (Windows/Linux) o Cmd (Mac) " +\
"para seleccionar múltiples datasets</i></p>"
    layout_principal = widgets.VBox([
        widgets.HTML(mensaje_titulo),
        widgets.HTML(mensaje_ayuda),
        selector,
        info_label,
        widgets.HTML("<hr>"),
        botones_control,
        widgets.HTML("<hr>"),
        resultado_area
    ])

    # muestra la interfaz
    display(layout_principal)

    ## función auxiliar
    
    def obtener_resultado():
        """
        Permite consultar después qué opciones se seleccionaron y procesaron
        """
        return resultado_final
    
    # devuelve la función para obtener el resultado
    return obtener_resultado

In [12]:
def save_dataframes(dataframes, directorio='datasets_covid'):
    """
    Guarda dataframes a disco en formato pkl, y un resumen en formato csv.

    Args:
        dataframes: diccionario con un conjunto de dataframes
        directorio: str, carpeta donde guardar los dataframes

    Returns:
        lista con un resumen de la información de los dataframes
    """
    # si la carpeta destino no existe la crea
    if not os.path.exists(directorio):
        os.makedirs(directorio)

    # obtiene la fecha actual en formato aaaa-mm-dd
    fecha_actual = datetime.now().strftime("%Y-%m-%d")
    
    # inicializa la lista con el resumen del proceso
    resumen = []

    # recorre los dataframes
    for nombre, info in dataframes.items():

        # usa la URI abreviada para el nombre del archivo
        
        #nombre_archivo = info['uri_abreviada']
        nombre_archivo = f"{info['uri_abreviada']}_{fecha_actual}"
        
        # guarda como pickle
        archivo_pickle = f"{directorio}/{nombre_archivo}.pkl"
        info['dataframe'].to_pickle(archivo_pickle)

        # guarda información del dataframe
        resumen.append({
            'nombre': nombre,
            'titulo': info['titulo'],
            'descripcion': info['descripcion'],
            'uri_abreviada': info['uri_abreviada'],
            #'archivo_csv': archivo_csv,
            'archivo_pickle': archivo_pickle,
            'filas': len(info['dataframe']),
            'columnas': len(info['dataframe'].columns)
        })
        
        print(f"✓ Guardado: {nombre_archivo}")
    
    # guarda un resumen del proceso
    resumen_nombre = f"resumen_datasets_{fecha_actual}.csv"
    df_resumen = pd.DataFrame(resumen)
    df_resumen.to_csv(f"{directorio}/{resumen_nombre}", index=False)
    
    print(f"\n📁 Todos los archivos guardados en: {directorio}/")
    print(f"📋 Resumen disponible en: {directorio}/{resumen_nombre}")
    
    return resumen

## 1. Carga de datos

In [13]:
# crea el gestor de datasets, manteniendo como máximo 3 datasets en memoria
manager = DatasetManager(cache_limit=3)

In [14]:
# busca datasets relacionados con COVID
datasets_df, num_public_datasets = manager.find_covid_datasets(
    filtro='covid', public_only=True
)

Total de datasets encontrados: 20159
Filtrando datasets que contienen 'covid'...
Datasets filtrados: 55
Total de datasets públicos: 43


In [15]:
# prueba de carga de los datasets
manager.test_datasets(max_count=num_public_datasets, timeout=30)


Probando datasets (con timeout de 30 segundos por dataset):
--------------------------------------------------
Probando dataset 5077 ✓ - Título: COVID-19, confirmed cases and deaths
 ✓ - URI única: data://garden/covid/latest/cases_deaths/cases_deaths
Probando dataset 5200 ✓ - Título: COVID-19, deaths by vaccination status
 ✓ - URI única: data://garden/covid/latest/deaths_vax_status/chile
Probando dataset 5632 ✓ - Título: COVID-19, combined indicators
 ✓ - URI única: data://garden/covid/latest/combined/combined
Probando dataset 5662 ✓ - Título: Global Health Estimates
 ✓ - URI única: data://garden/covid/latest/compact/compact
Probando dataset 6503 ✓ - Título: GitHub stats on owid/covid-19-data repository
 ✓ - URI única: data://garden/covid/2024-11-05/github_stats/contributions
Probando dataset 6524 ✓ - Título: COVID-19, Countries reporting data
 ✓ - URI única: data://garden/covid/latest/countries_reporting/country_counts
Probando dataset 6525 ✓ - Título: COVID-19, Government Response T

In [16]:
# obtiene la lista de referencias a datasets cargados ccorrectamente
successful_refs = manager.get_successful_refs()

print(f"\nTotal de datasets cargados correctamente: {len(successful_refs)}")
print(f"IDs de datasets cargados correctamente: {successful_refs}")


Total de datasets cargados correctamente: 43
IDs de datasets cargados correctamente: [5077, 5200, 5632, 5662, 6503, 6524, 6525, 6526, 6527, 6528, 6543, 6544, 6549, 6696, 7700, 7892, 9289, 9290, 9326, 9740, 9741, 10151, 10152, 10153, 10154, 11244, 11626, 16159, 18711, 18713, 18808, 18816, 19186, 19444, 19747, 19762, 19808, 19811, 19812, 19813, 20094, 20125, 20126]


In [17]:
# explora la estructura de 3 datasets
process_datasets(manager, max_datasets=3)

Procesando 3 datasets de 43 cargados correctamente:

--- Dataset 1/3: 5077 ---
Cargando dataset 5077... ✓
Título: COVID-19, confirmed cases and deaths
Tipo de objeto: <class 'owid.catalog.tables.Table'>

URI única: data://garden/covid/latest/cases_deaths/cases_deaths
URI abreviada: cases_deaths__cases_deaths

Metadatos:
  - Título: COVID-19, confirmed cases and deaths
  - Descripción: Daily COVID-19 cases and deaths by date reported to WHO.

From the 31 December 2019 to the 21 March ...

Estructura de datos:
  - Columnas: Index(['new_cases', 'total_cases', 'new_deaths', 'total_deaths',
       'weekly_cases', 'weekly_deaths', 'weekly_pct_growth_cases',
       'weekly_pct_growth_deaths', 'biweekly_cases', 'biweekly_deaths',
       'biweekly_pct_growth_cases', 'biweekly_pct_growth_deaths',
       'new_cases_per_million', 'new_deaths_per_million',
       'total_cases_per_million', 'total_deaths_per_million',
       'weekly_cases_per_million', 'weekly_deaths_per_million',
       'biweekly_c

In [18]:
# explora un dataset específico
if successful_refs:
    # selecciona el primer dataset cargado correctamente
    dataset_id = successful_refs[0]
    
    # carga el dataset
    dataset = manager.get_dataset(dataset_id)
    
    if dataset is not None:
        # explora la estructura para entender cómo acceder a los datos
        manager.explore_dataset_structure(dataset)
        
        # libera la memoria
        manager.clear_cache()

Cargando dataset 5077... ✓
Tipo de objeto: <class 'owid.catalog.tables.Table'>

URI única: data://garden/covid/latest/cases_deaths/cases_deaths
URI abreviada: cases_deaths__cases_deaths

Metadatos:
  - Título: COVID-19, confirmed cases and deaths
  - Descripción: Daily COVID-19 cases and deaths by date reported to WHO.

From the 31 December 2019 to the 21 March ...

Estructura de datos:
  - Columnas: Index(['new_cases', 'total_cases', 'new_deaths', 'total_deaths',
       'weekly_cases', 'weekly_deaths', 'weekly_pct_growth_cases',
       'weekly_pct_growth_deaths', 'biweekly_cases', 'biweekly_deaths',
       'biweekly_pct_growth_cases', 'biweekly_pct_growth_deaths',
       'new_cases_per_million', 'new_deaths_per_million',
       'total_cases_per_million', 'total_deaths_per_million',
       'weekly_cases_per_million', 'weekly_deaths_per_million',
       'biweekly_cases_per_million', 'biweekly_deaths_per_million',
       'total_deaths_per_100k', 'new_deaths_per_100k',
       'new_cases_7

## 2. Estudio de los datos

### 2.1. Análisis de los datasets

In [19]:
analisis = analyze_all_datasets(successful_refs, manager)

Analizando todos los datasets disponibles...
Total de datasets: 43

🔍 DATASET 1
Cargando dataset 5077... ✓
-------------------- URI ÚNICA --------------------
data://garden/covid/latest/cases_deaths/cases_deaths
URI ABREVIADA: cases_deaths__cases_deaths
-------------------- INFO --------------------
2025-07-01 12:48:50 [warning  ] Different values of 'unit' detected among variables: {'cases', 'deaths', 'deaths per million people', 'days', 'cases per million people', 'deaths per 100,000 people', '%'} category=DifferentValuesWarning filename=C:\Users\Enrique\anaconda3\envs\analisis-covid\Lib\site-packages\owid\catalog\variables.py lineno=360
<class 'owid.catalog.tables.Table'>
MultiIndex: 493767 entries, ('Afghanistan', Timestamp('2020-01-04 00:00:00')) to ('Zimbabwe', Timestamp('2025-06-08 00:00:00'))
Data columns (total 38 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   new_case

None

-------------------- COLUMNS --------------------


Index(['new_cases', 'total_cases', 'new_deaths', 'total_deaths',
       'weekly_cases', 'weekly_deaths', 'weekly_pct_growth_cases',
       'weekly_pct_growth_deaths', 'biweekly_cases', 'biweekly_deaths',
       'biweekly_pct_growth_cases', 'biweekly_pct_growth_deaths',
       'new_cases_per_million', 'new_deaths_per_million',
       'total_cases_per_million', 'total_deaths_per_million',
       'weekly_cases_per_million', 'weekly_deaths_per_million',
       'biweekly_cases_per_million', 'biweekly_deaths_per_million',
       'total_deaths_per_100k', 'new_deaths_per_100k',
       'new_cases_7_day_avg_right', 'new_deaths_7_day_avg_right',
       'new_cases_per_million_7_day_avg_right',
       'new_deaths_per_million_7_day_avg_right',
       'new_deaths_per_100k_7_day_avg_right', 'cfr', 'cfr_100_cases',
       'cfr_short_term', 'days_since_100_total_cases',
       'days_since_5_total_deaths', 'days_since_1_total_cases_per_million',
       'days_since_0_1_total_deaths_per_million',
       'd

-------------------- METADATA --------------------


...

-------------------- METADATA --------------------


{'short_name': 'yougov_composite',
 'title': '',
 'description': 'This is the data repository for the Imperial College London YouGov Covid 19 Behaviour Tracker Data Hub.\n\nYouGov has partnered with the Institute of Global Health Innovation (IGHI) at Imperial College London to gather global insights on people’s behaviours in response to COVID-19.\n\nIt is designed to provide behavioural analysis on how different populations are responding to the pandemic, helping public health bodies in their efforts to limit the impact of the disease. Anonymised respondent level data will be available for all public health and academic institutions globally.\n\nThe questions in the survey, led by IGHI, cover data on testing, symptoms, self-isolating in response to symptoms and the ability and willingness to self-isolate if needed. It also looks at behaviours, including going outdoors, working outside the home, contact with others, hand washing and the extent of compliance with 20 common preventative m

(172, 11)


,country,date,people_vaccinated_per_hundred,uncertain_covid_vaccinate_this_week_pct_pop,unwillingness_covid_vaccinate_this_week_pct_pop,willingness_covid_vaccinate_this_week_pct_pop,dataset_title,dataset_short_name,dataset_description,dataset_version,dataset_update_period_days
0,Australia,2021-03-15,0.71,19.799999,27.719999,51.77,"COVID-19, Behaviour Tracker Data Hub",yougov,This is the data repository for the Imperial C...,latest,0
1,Australia,2021-04-15,4.82,20.129999,31.32,43.73,"COVID-19, Behaviour Tracker Data Hub",yougov,This is the data repository for the Imperial C...,latest,0
2,Australia,2021-05-15,10.7,20.709999,31.139999,37.450001,"COVID-19, Behaviour Tracker Data Hub",yougov,This is the data repository for the Imperial C...,latest,0
3,Australia,2021-06-15,20.0,18.91,28.450001,32.639999,"COVID-19, Behaviour Tracker Data Hub",yougov,This is the data repository for the Imperial C...,latest,0
4,Australia,2021-07-15,27.57,15.7,25.690001,31.040001,"COVID-19, Behaviour Tracker Data Hub",yougov,This is the data repository for the Imperial C...,latest,0


-------------------- RESUMEN DE LOS DATOS --------------------
📊 Título principal: COVID-19, Behaviour Tracker Data Hub
📈 Título secundario: Desc: This is the data repository for the Imperial College London YouGov Covid 19 Behaviour Tracker Data H...
📏 Dimensiones: 172 filas × 11 columnas
📅 Columnas de fecha: date, dataset_update_period_days
🗓️  Rango temporal: 2020-12-15  → 2022-02-15
🏷️  Columnas: country, date, people_vaccinated_per_hundred, uncertain_covid_vaccinate_this_week_pct_pop, unwillingness_covid_vaccinate_this_week_pct_pop, willingness_covid_vaccinate_this_week_pct_pop, dataset_title, dataset_short_name ...

📋 Vista previa (primeras 3 filas):
     country       date  people_vaccinated_per_hundred  ...  dataset_description  dataset_version  dataset_update_period_days
0  Australia 2021-03-15                 0.71            ...  This is the data...           latest                    0       
1  Australia 2021-04-15                 4.82            ...  This is the data...    

<b>Resumen del análisis</b>

### Estudio de los datasets: conclusión

*Datasets interesantes para este estudio*

- Dataset 13 "covid": el más completo; URI única (identificador único del dataset) "covid__covid"
- Dataset 43 "COVID-19, Behaviour Tracker Data Hub": único para análisis comportamental; URI "yougov__yougov_composite"
- Dataset 37 "COVID-19, vaccinations (broken down by age)": único para análisis demográfico; URI "vaccinations_age__vaccinations_age"
- Dataset 21 "COVID-2019 - Hospital & ICU": especializado en hospitalizaciones; URI "hospital_and_icu__hospital_and_icu"
- Dataset 32 "COVID testing time series data": especializado en pruebas; URI "testing_time_series__testing_time_series"
- Dataset 38 "Population - COVID-19, vaccinations": especializado en vacunación avanzada; URI "vaccinations_global__vaccinations_global"
- Dataset 41 "Global excess deaths associated with COVID-19 (modelled estimates)": especializado en exceso de mortalidad; URI "xm_who__xm_who"
- Dataset 15 "COVID-2019 - ECDC (2020)": algunos datos solapados, pero aporta un análisis temporal único; URI "ecdc__ecdc"
- Dataset 17 "Google Mobility Reports": movilidad de las personas; URI "google_mobility__google_mobility"
- Dataset 28 "Government Response Tracker (OxCGRT)": especializado en las políticas gubernamentales, más completo y estructurado que el dataset 19; URI "oxcgrt_policy__oxcgrt_policy"

*Datasets que no vamos a utilizar*

- Dataset 4 "compact": es una versión reducida del 13
- Dataset 27 "COVID-19, JHU CSSE Dataset": duplicado en su mayoría
- Dataset 1 "COVID-19, confirmed cases and deaths": datos básicos duplicados, aporta validación oficial
- Dataset 19 "COVID Government Response (OxBSG)": especializado en las políticas gubernamentales

*Lista de Datasets que vamos a usar en el estudio*

1,4,13,15,17,19,21,27,28,32,37,38,41,43

### 2.2. Selección de datasets

Seleccionamos sólo los datasets que hemos considerado que interesan para el estudio.

In [20]:
dataframes_seleccionados = select_datasets(analisis)

### 2.3. Guardado de los datasets

Guarda en disco los datasets que nos interesan, para utilizarlos en 
posteriores Notebooks.

In [21]:
resumen = save_dataframes(dataframes_seleccionados(), directorio=dataframe_path)

✓ Guardado: cases_deaths__cases_deaths_2025-07-01
✓ Guardado: compact__compact_2025-07-01
✓ Guardado: covid__covid_2025-07-01
✓ Guardado: ecdc__ecdc_2025-07-01
✓ Guardado: google_mobility__google_mobility_2025-07-01
✓ Guardado: government_response_oxbsg__government_response_oxbsg_2025-07-01
✓ Guardado: hospital_and_icu__hospital_and_icu_2025-07-01
✓ Guardado: john_hopkins_university__john_hopkins_university_2025-07-01
✓ Guardado: oxcgrt_policy__oxcgrt_policy_2025-07-01
✓ Guardado: testing_time_series__testing_time_series_2025-07-01
✓ Guardado: vaccinations_age__vaccinations_age_2025-07-01
✓ Guardado: vaccinations_global__vaccinations_global_2025-07-01
✓ Guardado: xm_who__xm_who_2025-07-01
✓ Guardado: yougov__yougov_composite_2025-07-01

📁 Todos los archivos guardados en: ..//data//dataframes/
📋 Resumen disponible en: ..//data//dataframes/resumen_datasets_2025-07-01.csv


In [22]:
print("Dataframes guardados:")
for item in resumen:
    print(f"• {item['nombre']}: {item['uri_abreviada']} ({item['filas']} filas)")

Dataframes guardados:
• df_1: cases_deaths__cases_deaths (493767 filas)
• df_4: compact__compact (508524 filas)
• df_13: covid__covid (429435 filas)
• df_15: ecdc__ecdc (64535 filas)
• df_17: google_mobility__google_mobility (784944 filas)
• df_19: government_response_oxbsg__government_response_oxbsg (202768 filas)
• df_21: hospital_and_icu__hospital_and_icu (46777 filas)
• df_27: john_hopkins_university__john_hopkins_university (253130 filas)
• df_28: oxcgrt_policy__oxcgrt_policy (202760 filas)
• df_32: testing_time_series__testing_time_series (106788 filas)
• df_37: vaccinations_age__vaccinations_age (74142 filas)
• df_38: vaccinations_global__vaccinations_global (203057 filas)
• df_41: xm_who__xm_who (4920 filas)
• df_43: yougov__yougov_composite (172 filas)
